# Create db

In [1]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from bs4 import BeautifulSoup
from langchain.llms import OpenAI

In [2]:
#from sentence_transformers import SentenceTransformer
#embeddings =  SentenceTransformer("dangvantuan/sentence-camembert-large")

In [3]:
def _preprocess_text(text: str) -> str:
        soup = BeautifulSoup(text, "html.parser").text
        soup_tab = soup.expandtabs()
        cleaned_text = ""

        for line in soup_tab.split("\n"):
            if line:
                cleaned_text += ". " + line

        return cleaned_text

In [4]:
def create_chunks(text, chunk_size=1000):
    # Create Document object from text documents 
    docs = [Document(page_content=post) for post in [text]]

    # Split documents into chunks 

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=10, separators=['\n\n', '\n', '.']
    )
    docs = text_splitter.split_documents(docs)
    return docs

## Store Embedings

In [5]:
def store_embeddings(docs):
    db = FAISS.from_documents(docs, embeddings)
    return db

# Prepare prompt

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
from langchain.prompts import PromptTemplate

template_extr = """Compte tenu des éléments d'information dans Contexte, récupérez la réponse à la question de l'utilisateur spécifié
Contexte : {context}

Question : {question}

Eviter de donner une opinion

"""

prompt_extr = PromptTemplate(template=template_extr,
                            input_variables=['context', 'question'])

In [8]:
#___prompt Titre______

template_titre = """
Créez un titre à la fois succinct et explicite pour résumer la décision judiciaire, en prenant en compte les éléments clés et le contexte suivant

Contexte : {context}

Le titre devrait capturer l'essence de la décision et les points principaux du contexte, tout en étant facilement compréhensible pour un public général, sans expertise juridique spécifique
"""

prompt_titre = PromptTemplate(template=template_titre, input_variables=['context'])

In [9]:
#___prompt Resume______

template_resume = """
Veuillez fournir un résumé succinct et précis de la décision judiciaire, en se basant sur les informations clés fournies dans le contexte suivant 

Contexte : {context}

Le resumé doit contenir au maximum 5 lignes

Assurez-vous que le résumé mette en avant les principaux points de la décision, les arguments clés des parties impliquées, et les implications légales de cette décision, tout en restant accessible à un public non spécialisé en droit.

"""

prompt_resume = PromptTemplate(template=template_resume, input_variables=['context'])


#___prompt Resume Patice______

template_resume_patrice = """
Veuillez fournir un résumé succinct et précis de la décision judiciaire, en se basant sur les informations clés fournies dans le contexte suivant 

Contexte : {context}

Expliquer le résumé à Patrice agé de 60 ans, dirigeant d'une PME sans expertise juridique ni expérience contentieuse.

Le resumé doit contenir au maximum 5 lignes

Assurez-vous que le résumé mette en avant les principaux points de la décision, les arguments clés des parties impliquées, et les implications légales de cette décision, tout en restant accessible à un public non spécialisé en droit.

Eviter de donner une opinion

"""

prompt_resume_patrice = PromptTemplate(template=template_resume_patrice, input_variables=['context'])

#___prompt Resume Hugo______

template_resume_hugo = """
Veuillez fournir un résumé succinct et précis de la décision judiciaire, en se basant sur les informations clés fournies dans le contexte suivant 

Contexte : {context}

Expliquer le résumé à Hugo, âgé de 20 ans, qui souffre de déficiences cognitives. Son handicap ne lui permet que de lire et comprendre le français facile à lire et à comprendre FALC. 


Le resumé doit contenir au maximum 5 lignes

Assurez-vous que le résumé mette en avant les principaux points de la décision, les arguments clés des parties impliquées, et les implications légales de cette décision, tout en restant accessible à un public non spécialisé en droit.

Eviter de donner une opinion
"""

prompt_resume_hugo = PromptTemplate(template=template_resume_hugo, input_variables=['context'])


#___prompt Resume Anna______

template_resume_anna = """
Veuillez fournir un résumé succinct et précis de la décision judiciaire, en se basant sur les informations clés fournies dans le contexte suivant 

Contexte : {context}

Expliquer le résumé à Anna, âgée de 30 ans, qui est réfugiée et vit en France depuis 2022. Elle apprend le français et à un niveau de langue A2.

Le resumé doit contenir au maximum 5 lignes

Assurez-vous que le résumé mette en avant les principaux points de la décision, les arguments clés des parties impliquées, et les implications légales de cette décision, tout en restant accessible à un public non spécialisé en droit.

Eviter de donner une opinion
"""

prompt_resume_anna = PromptTemplate(template=template_resume_anna, input_variables=['context'])

In [10]:
#___prompt Patrice______

template_patrice = """Expliquez le contexte suivant à Patrice agé de 60 ans, dirigeant d'une PME sans expertise juridique ni expérience contentieuse. 

Contexte : {context}

Rédiger une explication en langage clair et simple mais soutenu et sans utiliser des termes techniques. Inutile de le préciser à Patrice ni de faire mention de sa profession

Rentrer directement dans le vif de sujet sans introduction et conclusion

Il faut s'addresser directement à Patrice

S'agissant d'un document juridique le ton doit être formel

Eviter de donner une opinion
"""

prompt_patrice = PromptTemplate(template=template_patrice, input_variables=['context'])

In [11]:
#___prompt Anna______

template_anna_old = """Expliquez le contexte suivant à Anna, âgée de 30 ans, qui est réfugiée et vit en France depuis 2022. Elle apprend le français et à un niveau de langue A2.

Contexte : {context}

Rédiger une explication en langage clair et basique sans utiliser des termes techniques. Il faut lui expliquer entre parenthèse le vocabulaire juridique.  

 
Rentrer directement dans le vif de sujet sans introduction et conclusion

Structurer la réponse en ajoutant des puces et un titre en gras pour expliquer la portée de chaque paragraphe.


Il faut s'adresser directement à Anna.

Eviter de donner une opinion
"""

prompt_anna_old = PromptTemplate(template=template_anna_old, input_variables=['context'])

In [12]:
#___prompt Anna______

template_anna = """Expliquez le contexte suivant à Anna, âgée de 30 ans, qui est réfugiée et vit en France depuis 2022. Elle apprend le français et à un niveau de langue A2.

Contexte : {context}

Rédiger une explication en langage clair et basique sans utiliser des termes techniques. Il faut lui expliquer entre parenthèse le vocabulaire juridique.  

 
Rentrer directement dans le vif de sujet sans introduction et conclusion

Structurer la réponse en ajoutant des puces


Il faut s'adresser directement à Anna.

Eviter de donner une opinion
"""

prompt_anna = PromptTemplate(template=template_anna, input_variables=['context'])

In [13]:
#___prompt Hugo______

template_hugo = """
Expliquez le contexte suivant à Hugo, âgé de 20 ans, qui souffre de déficiences cognitives. Son handicap ne lui permet que de lire et comprendre le français facile à lire et à comprendre FALC. 
Il faut lui expliquer le contexte sans utiliser des termes juridiques mais en gardant le nom des juridictions.

Contexte : {context}

Rédiger une explication de dix lignes maximum en français facile à lire et à comprendre. Inutile de le préciser à Hugo ni de faire mention de son handicap.

Rentrer directement dans le vif de sujet sans introduction et conclusion 

Structurer la réponse en ajoutant des puces.


Il faut s'adresser directement à Hugo.

Eviter de donner une opinion
"""

prompt_hugo = PromptTemplate(template=template_hugo, input_variables=['context'])

In [14]:
persona = {
    "Patrice": prompt_patrice,
    "Anna":prompt_anna,
    "Hugo":prompt_hugo
}

# Preparation du model

In [15]:
import os
os.environ["OPENAI_API_KEY"] = ""
#engine = "gpt-3.5-turbo"
engine = "gpt-4-1106-preview"
llm = OpenAI(model_name=engine, temperature=0.1)

C:\Users\MahamanSanoussiYahay\anaconda3\envs\hackathon1\lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\MahamanSanoussiYahay\anaconda3\envs\hackathon1\lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


# Définition des fonctions d'extraction

In [16]:
def retrieve(db, vector_count, query):
    retriever=db.as_retriever(search_kwargs={'k': vector_count})
    docs = retriever.get_relevant_documents(query)
    return ". ".join([doc.page_content for doc in docs])

In [17]:
from langchain.chains import LLMChain
#qa_extr = LLMChain(llm=llm, prompt=prompt_extr)

In [18]:
# qa_exp = LLMChain(llm=llm, prompt=prompt_exp)

In [19]:
def extract(text, query):
    qa_extr = LLMChain(llm=llm,
            prompt=prompt_extr)
    
    input_list = [{"context": text, "question": query}]
    output = qa_extr.apply(input_list)
    if isinstance(output, list):
        if isinstance(output[0], dict):
            return ". ".join(list(output[0].values()))
        else:
            return output[0]
    else:
        return output

In [20]:
def explication(text, prompt):
    qa_exp = LLMChain(llm=llm, prompt=prompt)
    
    input_list = [{"context": text}]
    output = qa_exp.apply(input_list)

    if isinstance(output, list):
        if isinstance(output[0], dict):
            return ". ".join(list(output[0].values()))
        else:
            return output[0]
    else:
        return output

In [21]:
def extract_decision_component(input_string, start_line, end_line=None):
    start_index = input_string.find(start_line)
    if start_index == -1:
        return "Start line not found in the input string."

    start_index += len(start_line)

    if end_line is not None:
        end_index = input_string.find(end_line, start_index)
        if end_index == -1:
            return "End line not found in the input string after the start line."
    else:
        end_index = len(input_string)

    return input_string[start_index:end_index].strip()

In [22]:
text_decision = """


COUR ADMINISTRATIVE D'APPEL 
DE LYON

N° 19LY01693
__________

M. G... A...
__________

Mme Rozenn B...
Rapporteure
__________

Mme Cécile Cottier
Rapporteure publique
__________

Audience du 25 mars 2021 
Décision du 16 avril 2021 
__________

60 02 01 01 005
C	





RÉPUBLIQUE FRANÇAISE



AU NOM DU PEUPLE FRANÇAIS



La cour administrative d'appel de Lyon

6ème chambre




Vu la procédure suivante :

Procédure contentieuse antérieure :

M. G... A..., Mme D... A... et M. H... A... ont demandé au tribunal administratif de Clermont Ferrand de condamner le centre hospitalier universitaire de Clermont Ferrand à verser à M. G... A... la somme de 203 233,75 euros, à Mme D... A... et à M. H... A..., la somme de 2 250 euros chacun, au titre de leurs préjudices subis du fait de l’infection nosocomiale contractée par M. G... A... lors de sa prise en charge par le centre hospitalier universitaire et des fautes médicales commises dans l’administration du traitement antibiotique dans ce même établissement, ces sommes devant être assorties des intérêts au taux légal et de la capitalisation des intérêts.

La caisse primaire d’assurance maladie de la Haute Garonne, appelée à l’instance, a demandé au tribunal administratif de Clermont Ferrand de condamner le centre hospitalier universitaire de Clermont Ferrand à lui verser la somme totale de 213 154,89 euros en remboursement des débours engagés et futurs, sous déduction de la provision déjà réglée, somme assortie des intérêts au taux légal et de la capitalisation des intérêts, et la somme de 1 066 euros au titre de l’indemnité forfaitaire de gestion prévue par l’article L. 376 1 du code de la sécurité sociale.


Par un jugement n° 1601748 du 6 mars 2019, le tribunal administratif de Clermont Ferrand a condamné le centre hospitalier universitaire de Clermont Ferrand à verser à M. G... A... la somme de 63 816 euros en réparation de ses préjudices, sous déduction de la provision de 43 585,50 euros déjà versée par le centre hospitalier en exécution de l’ordonnance du juge des référés du tribunal administratif de Clermont Ferrand du 21 avril 2015, soit la somme résiduelle de 20 230,50 euros, somme assortie des intérêts au taux légal et de la capitalisation des intérêts, à verser à Mme D... A... et M. H... A..., en réparation de leurs préjudices, la somme de 1 500 euros chacun, correspondant au montant des provisions de 1 500 euros qui leur ont déjà été versées par le centre hospitalier universitaire en exécution de l’ordonnance du juge des référés du 21 avril 2015, ces sommes étant assorties des intérêts au taux légal et de la capitalisation des intérêts, à verser à la caisse primaire d’assurance maladie de la Haute-Garonne la somme de 109 871 euros en remboursement des débours exposés et des frais de reclassement professionnel, sous déduction de la provision de 82 780 euros déjà versée par le centre hospitalier universitaire en exécution de l’ordonnance du 21 avril 2015, soit la somme résiduelle de 27 091 euros, sommes assorties des intérêts au taux légal et de la capitalisation des intérêts, une rente annuelle d’un montant de 1 691 euros au titre des frais d’appareillage, cette rente étant revalorisée en tenant compte du prix de l’appareil principal au jour du versement et la somme de 1 080 euros au titre de l’indemnitaire forfaitaire de gestion et a rejeté le surplus des conclusions des parties.

Procédure devant la cour :

Par une requête, enregistrée le 26 avril 2019, et un mémoire complémentaire, enregistré le 17 juin 2020, M. G... A..., représenté par Me C..., demande à la cour : 

1°) de réformer le jugement du 6 mars 2019 en tant que le tribunal administratif de Clemont Ferrand n’a pas fait droit à la totalité de ses demandes indemnitaires concernant la perte de gains professionnels actuels, l’incidence professionnelle et les souffrances endurées ; 

2°) de condamner le centre hospitalier universitaire de Clermont Ferrand à lui verser la somme de 202 465 euros dont 95 323 euros en réparation de sa perte de gains professionnels actuels, 11 250 euros en réparation des souffrances endurées et 80 892 euros en réparation de l’incidence professionnelle, outre les intérêts au taux légal et la capitalisation des intérêts ; 

3°) de mettre à la charge du centre hospitalier universitaire de Clermont-Ferrand la somme de 3 000 euros en application des dispositions de l’article L. 761 1 du code de justice administrative et les entiers dépens. 

Il soutient que : 
– il n’est pas contesté qu’il a contracté une infection nosocomiale lors de son séjour au centre hospitalier universitaire de Clermont Ferrand et qu’il appartient à l’établissement de soins de réparer les dommages résultant de cette infection ; 
– s’agissant de la perte de gains professionnels actuels, lors de la survenance de l’infection nosocomiale, il était âgé de 17 ans et était employé en qualité d’apprenti pâtissier ; il n’a pas pu reprendre son apprentissage et a dû être réorienté ; il a été inscrit à Pôle emploi, a suivi une remise à niveau et enfin une formation qualifiante d’ouvrier en prothèse dentaire à compter du 16 août 2012 ; durant sa période d’inactivité, il a subi sept années de déficit fonctionnel temporaire, neuf interventions chirurgicales, dix hospitalisations en établissement et une hospitalisation à domicile ; le tribunal administratif ne pouvait considérer qu’il n’a pas pu reprendre sa formation d’apprenti et décider qu’il n’a pas été privé des revenus correspondants durant la période d’apprentissage et des revenus que cette formation et cette profession devaient lui assurer durant les années suivantes ; il ne saurait être considéré que le contrat a été rompu, et non simplement suspendu, par la seule volonté de ses parents ; l’infection nosocomiale contractée au centre hospitalier universitaire de Clermont Ferrand et sa prise en charge défaillante ont lourdement obéré son avenir mais l’ont aussi privé de tous gains professionnels pendant de nombreuses années ; pour la période d’apprentissage allant du 12 juillet 2004 au 11 juillet 2006, il sollicite la somme de 9 852,59 euros ; postérieurement à la période de formation, il aurait pu prétendre à un poste d’ouvrier catégorie 2 coefficient 165 selon le tableau de la convention collective nationale et percevoir la somme de 98 703,21 euros ; au 1er août 2012, il a pu intégrer la formation pour adulte handicapé de prothésiste dentaire ; pendant sa période de formation au métier d’auxiliaire en prothèse dentaire, il a perçu la somme de 602,02 euros par mois en lieu et place de la rémunération qu’il aurait dû percevoir s’il avait pu poursuivre sa formation de pâtissier et le manque à gagner s’élève à  la somme de 18 541,55 euros ; durant une période de neuf années, le total des rémunérations non perçues au titre du contrat d’apprentissage et de l’emploi qu’il aurait pu occuper s’élève donc à la somme de 127 097,35 euros ; par suite, il y a lieu de mettre à la charge du centre hospitalier universitaire de Clermont Ferrand la somme de 95 323 euros après application du taux de perte de chance retenu de 75 % ; son contrat d’apprentissage expirait le 12 juillet 2006 et non le 12 juillet 2005 ; ce contrat aurait été renouvelé ainsi qu’en atteste son employeur ; 
– s’agissant des souffrances endurées, les experts ont évalué ces souffrances à 6 sur une échelle de 7 ; compte tenu de l’intensité des souffrances endurées, de leur durée, de son jeune âge, il sollicite la somme de 35 000 euros, soit compte tenu du taux de perte de chance retenu de 75 %, la somme de 26 250 euros ; 
– s’agissant de l’incidence professionnelle, il subit une importante dévalorisation sur le marché du travail due aux séquelles définitives de l’infection nosocomiale et de ses suites ; il est reconnu travailleur handicapé ; la formation qualifiante n’a pas permis de compenser cette incidence professionnelle ; cette formation s’avère inadaptée puisqu’il a été récemment licencié pour inaptitude avec une impossibilité de reclassement ; l’évaluation du tribunal administratif de ce préjudice à la somme de 38 000 euros est insuffisante ; il n’y a pas lieu de déduire la somme de 17 583,81 euros qui a été versée par la caisse primaire d’assurance maladie de Haute Garonne au centre de rééducation des invalides civils (CRIC) ; la somme de 80 892 euros tenant compte du taux de perte de chance retenu doit lui être allouée au titre de l’incidence professionnelle ; l’incidence professionnelle concerne les jeunes en formation dont le préjudice sur la carrière est indemnisé au titre de la perte de chance professionnelle ; le mécanisme de la subrogation prévu par l’article L. 376 1 du code de la sécurité sociale ne peut avoir pour effet de diminuer l’indemnisation versée à la victime ; lui imputer le montant de cette formation reviendrait à lui faire supporter le coût de cette formation.

Par des mémoires, enregistrés les 4, 12 et 19 juin 2020, le centre hospitalier universitaire de Clermont-Ferrand, représenté par Me F..., conclut au rejet de la requête. 

Il soutient que : 
– le calcul des pertes de revenus s’établit en fonction du salaire perçu par la victime au jour de l’accident et il est évalué à la différence entre le salaire qu’aurait dû percevoir l’intéressé et les revenus effectivement perçus ; si, à la date à laquelle l’infection s’est déclarée, le patient n’avait aucun contrat de travail, aucune somme n’est due au titre de la perte de gains professionnels ; M. A... a débuté son contrat d’apprentissage le 12 juillet 2004 et son accident date du 4 septembre de la même année ; le 6 septembre 2004, le père de M. A... a écrit à l’institut des métiers aux fins d’annuler l’inscription de son fils en première année de CAP et a écrit à l’employeur de son fils pour lui préciser qu’il était raisonnable de ne pas confirmer son contrat d’apprentissage ; à la date du 6 septembre 2004, M. A... n’avait été victime d’aucune infection nosocomiale ; c’est uniquement l’accident initial qui a justifié la rupture de son contrat de travail et à la date de l’infection nosocomiale, il ne disposait d’aucun contrat de travail et n’a donc pas été victime d’une perte de gains professionnels en lien avec cette infection nosocomiale ; en tout état de cause, il n’est pas fondé à solliciter un préjudice résultant de la perte de revenus professionnels distinct de celui réparé au titre des troubles dans ses conditions d’existence ; si la cour venait à juger que M. A... justifie d’une perte de gains professionnels, il n’aurait droit tout au plus qu’à une indemnité courant du 12 juillet 2004 au 12 juillet 2005 et il conviendrait de prendre en compte la durée d’indisponibilité en lien avec l’accident qui aurait duré six mois ; 
– il convient de prendre en compte la durée d’indisponibilité en lien avec son accident dont M. A... relève qu’elle aurait duré six mois ; M. A... n’établit pas que le contrat allait être renouvelé ; 
– M. A... n’a pas été privé de toute possibilité d’exercer un jour une activité professionnelle ; 
– il n’est pas fondé à solliciter l’indemnisation de l’incidence professionnelle évaluée sur la base d’un revenu théorique auquel il appliquerait le taux de déficit fonctionnel permanent dont il reste atteint ; l’incidence professionnelle est évaluée de façon forfaitaire et non en fonction du salaire perçu ; la somme allouée par le tribunal est conforme voire supérieure à ce qu’alloue traditionnellement la jurisprudence ; c’est à bon droit que le tribunal administratif a déduit de l’indemnité allouée à M. A... l’indemnité versée par la caisse au titre de la reconversion professionnelle de M. A... ; il n’établit pas que cette indemnité ne lui a pas été versée ; 
– M. A... n’est pas fondé à soutenir que l’évaluation de son pretium doloris par le tribunal administratif serait insuffisante ; 
– s’agissant de la créance de la caisse primaire d’assurance maladie, aucune conclusion n’ayant été dirigée contre la caisse, laquelle conclut à la confirmation du jugement, elle n’avait aucune obligation de conclure devant la cour ; il ne saurait être fait droit aux conclusions tendant à sa condamnation au paiement des frais irrépétibles de la caisse ; la caisse ne peut prétendre en appel à sa condamnation au paiement de l’indemnité forfaitaire de gestion dès lors que cette dernière n’est allouée qu’une fois pour l’ensemble de la procédure ; la caisse primaire d’assurance maladie ne justifie pas que l’état de santé de M. A... justifie, du fait de son handicap causé par l’infection nosocomiale, des séances de kinésithérapie tout au long de sa vie.

Par un mémoire, enregistré le 8 juin 2020, la caisse primaire d’assurance maladie de la Haute-Garonne, représentée par la SCP Vinsonneau Paliès Noy Gauer et associés, conclut à la confirmation du jugement du 6 mars 2019 du tribunal administratif de Clermont Ferrand, à la condamnation du centre hospitalier universitaire de Clermont Ferrand à lui verser la somme de 109 871 euros avec intérêts de droit à compter du 30 avril 2014 et une rente annuelle de 1 691 euros qui sera revalorisée en tenant compte du prix de l’appareil principal au jour du versement en remboursement de ses débours et la somme de 1 091 euros au titre de l’indemnité forfaitaire de gestion prévue par les dispositions de l’article L. 376 1 du code de la sécurité sociale et à ce que la somme de 1 000 euros soit mise à la charge du centre hospitalier universitaire de Clermont Ferrand en application des dispositions de l’article L. 761 1 du code de justice administrative. 

Elle soutient que : 
– les dépenses de santé actuelles sont au moins égales à la somme de 128 910,17 euros dûment justifiée et augmentée des demandes de M. A... ; ces demandes ne pourront en aucun cas venir en diminution de celles de la caisse ; 
– sa créance définitive correspondant au poste « dépenses de santé futures » doit s’imputer exclusivement sur le montant de l’indemnisation susceptible d’être allouée au même titre à M. A... ; la créance de dépenses de santé futures est au moins égale à la somme de 63 794,72 euros dûment justifiée et prise en charge par la caisse ; 
– elle est fondée à solliciter la condamnation du centre hospitalier universitaire au paiement de la somme de 20 450 euros au titre des frais de reclassement professionnel servis pour le compte de M. A... et les demandes de ce dernier ne pourront pas venir en diminution de celles de la caisse.

Vu les autres pièces du dossier ;

Vu :
– le code de la santé publique ;
– le code de la sécurité sociale ;  
– le code de justice administrative.


Les parties ont été régulièrement averties du jour de l’audience.

Ont été entendus au cours de l’audience publique :
– le rapport de Mme B..., 
– les conclusions de Mme Cottier, rapporteure publique ; 
–  et les observations de Me C..., représentant M. A... et celles de Me E..., substituant Me F..., représentant le centre hospitalier universitaire de Clermond Ferrand.


Considérant ce qui suit :


1.	Le 4 septembre 2004, M. G... A..., né le 22 mars 1987, a été victime, alors qu’il circulait en scooter, d’un accident de la circulation qui lui a occasionné une double fracture fermée du fémur droit. Transporté au centre hospitalier universitaire de Clermont Ferrand, il a subi, le 5 septembre 2004, une ostéosynthèse consistant dans la pose d’un clou centro médullaire et d’une plaque supra condylienne et un traitement antibiotique lui a été administré. Le 13 septembre 2004, M. A... a présenté de la fièvre, des fortes douleurs puis, à la fin du mois de septembre 2004, une fistulisation à l’extrémité inférieure de la cicatrice sur la face externe. Des prélèvements réalisés le 11 octobre 2004 ont mis en évidence la présence d’un staphylococcus aureus oxacilline. Le 5 novembre 2004, M. A... a subi une reprise chirurgicale au centre hospitalier universitaire de Clermont Ferrand. Devant la persistance des troubles, M. A... a consulté le docteur Beal, chirurgien orthopédiste exerçant à la clinique « Pôle Santé République » de Clermont Ferrand qui a procédé, le 23 décembre 2004, à l’ablation de la plaque et au nettoyage de l’extrémité inférieure du fémur et a prescrit une antibiothérapie. Le 3 mars 2005, le docteur Beal a procédé à l’ablation du clou centro médullaire du fémur droit et au curetage au niveau de l’ostéite et a prescrit une antibiothérapie compte tenu de la persistance du phénomène infectieux. Le 9 mars 2005, le docteur Beal a constaté l’impotence fonctionnelle du membre inférieur droit liée à une fracture de son foyer diaphysaire fémoral et a prescrit à M. A... un alitement complet pour une durée d’un mois. Compte tenu de la présence d’une ostéite chronique, le 26 septembre 2005, M. A... a subi une nouvelle intervention consistant en une excision de la fistule, un curetage et un nettoyage du site. Inquiet de l’évolution de son état de santé, M. A... a consulté le service de chirurgie osseuse et traumatologique du groupe hospitalier Diaconesses Croix Saint Simon situé à Paris où il a subi un nouveau curetage, un comblement au ciment, une ostéosynthèse par plaque vissée, une reprise du cal vicieux, une décortication et une réduction. Les prélèvements opératoires effectués ont mis en évidence la présence d’un staphylococcus aureus avec de multiples souches. Le 24 mai 2006, M. A... a subi une nouvelle ostéosynthèse et un apport osseux. Le 24 novembre 2010, il a été réopéré pour un changement du matériel d’ostéosynthèse. Le 22 mars 2011, victime d’une fracture spontanée du fémur droit, M. A... a subi une nouvelle intervention à l’hôpital Purpan situé à Toulouse. 

2.	Estimant avoir été victime d’une infection nosocomiale et d’une défaillance dans sa prise en charge au centre hospitalier universitaire de Clermont-Ferrand, le 25 juin 2012, M. A... a saisi la commission régionale de conciliation et d’indemnisation des accidents médicaux, des affections iatrogènes et des infections nosocomiales (CRCI) d’Auvergne d’une demande d’indemnisation des préjudices subis. La commission de conciliation et d’indemnisation a ordonné une expertise confiée au docteur Lignac, chirurgien orthopédiste, et au professeur Massip, spécialiste des maladies infectieuses. A la suite du dépôt du rapport d’expertise le 18 octobre 2012, la commission de conciliation et d’indemnisation a estimé, dans un avis du 10 janvier 2013, que le centre hospitalier universitaire de Clermont-Ferrand était responsable à hauteur de 75 % des préjudices subis par M. A... qui résultaient d’une infection nosocomiale et de fautes commises lors de sa prise en charge et qu’il appartenait à la société hospitalière d’assurances mutuelles, l’assureur du centre hospitalier, de faire une offre d’indemnisation. L’assureur du centre hospitalier universitaire a formulé une offre d’indemnisation à hauteur de 45 927,20 euros laquelle a été rejetée par les consorts A.... Par une ordonnance du 21 avril 2015, le juge des référés du tribunal administratif de Clermont Ferrand a condamné le centre hospitalier universitaire de Clermont Ferrand à verser à M. A... une provision de 41 335,50 euros au titre des préjudices extrapatrimoniaux, de 2 250 euros au titre de son préjudice de formation et à verser au père et à la mère de M. A... une provision de 1 500 euros chacun. M. A... relève appel du jugement du 6 mars 2019 en tant que le tribunal administratif de Clermont Ferrand n’a pas fait droit à la totalité de ses demandes indemnitaires concernant la perte de gains professionnels actuels, l’incidence professionnelle et les souffrances endurées. 

Sur la responsabilité du centre hospitalier universitaire de Clermont Ferrand :

3.	Aux termes du I de l’article L. 1142 1 du code de la santé publique : « Hors le cas où leur responsabilité est encourue en raison d'un défaut d'un produit de santé, les professionnels de santé mentionnés à la quatrième partie du présent code, ainsi que tout établissement, service ou organisme dans lesquels sont réalisés des actes individuels de prévention, de diagnostic ou de soins ne sont responsables des conséquences dommageables d'actes de prévention, de diagnostic ou de soins qu'en cas de faute. / Les établissements, services et organismes susmentionnés sont responsables des dommages résultant d'infections nosocomiales, sauf s'ils rapportent la preuve d'une cause étrangère ». Aux termes de l’article L. 1142 1 1 du même code : « Sans préjudice des dispositions du septième alinéa de l'article L. 1142 17, ouvrent droit à réparation au titre de la solidarité nationale : / 1° Les dommages résultant d'infections nosocomiales dans les établissements, services ou organismes mentionnés au premier alinéa du I de l'article L. 1142 1 correspondant à un taux d'atteinte permanente à l'intégrité physique ou psychique supérieur à 25 % déterminé par référence au barème mentionné au II du même article, ainsi que les décès provoqués par ces infections nosocomiales (…) ». Ces dispositions font peser sur l’établissement de santé la responsabilité des dommages résultant des infections nosocomiales ayant entrainé un taux d’incapacité permanente inférieur ou égal à 25 %, qu’elles soient exogènes ou endogènes et à condition que l’infection survenant au cours ou au décours d’une prise en charge n’ait été ni présente, ni en incubation au début de cette prise en charge, sauf s'il est établi qu'elle a une autre origine que la prise en charge.

4.	D’une part, il résulte de l’instruction et notamment du rapport d’expertise du professeur Massip et du docteur Lignac que la fracture de M. A... a été traitée par ostéosynthèse le 5 septembre 2004. « Les premiers signes évocateurs d’infection sont apparus dès le 13 septembre et devenus francs le 19 septembre marqués par une hyperthermie à 39°. Le 11 octobre, l’écoulement de pus au niveau d’une déhiscence cicatricielle permet la mise en évidence d’un staphylocoque doré. (…) Les soins en rapport avec cette infection (…) se sont étendus du 19 septembre 2004 au 1er octobre 2011, ont comporté 10 hospitalisations en centre de santé et une hospitalisation à domicile, 9 interventions chirurgicales. (…) Le dommage est en lien direct avec l’infection du foyer opératoire. Il s’agit d’une infection nosocomiale. ». Les experts retiennent, sans que cela ne soit contesté par le centre hospitalier universitaire de Clermont Ferrand, le caractère nosocomial de cette infection. Il ne résulte pas de l’instruction que cette infection était présente ou en incubation au début de la prise en charge ou qu’elle procèderait d’une autre origine. Par suite, il y a lieu de confirmer l’appréciation des premiers juges sur ce point. 

5.	D’autre part, il résulte de l’instruction et notamment du rapport d’expertise, et il n’est pas contesté, que les experts ont relevé que « Les premiers signes de l’infection sont apparus dès le 13 septembre et devenus évidents le 19 et le 20 septembre soit au 15ème jour de l’intervention. La prise en charge chirurgicale a eu lieu le 5 novembre, soit deux mois plus tard. Le diagnostic évoqué le 19 septembre aurait dû être rapidement confirmé et conduire à une reprise chirurgicale traitant l’infection au plus tard dans les huit jours suivants. Tout retard conduit à une aggravation en particulier des lésions osseuses dont l’évolution se fait vers la chronicité et des lésions définitives. Le geste chirurgical, en raison du caractère tardif, aurait dû comporter l’ablation du matériel interne et mise en place d’une ostéosynthèse externe. Le traitement antibiotique prescrit est correct pour le choix des molécules, insuffisant quant à la dose de Bristopen et la voie d’administration per os (mauvaise bio disponibilité). Dans ce type de situation, il est recommandé d’utiliser une antibiothérapie intraveineuse à forte dose. Ces faits sont constitutifs d’un manquement aux règles et à l’origine d’une perte de chance ». Par suite, les manquements relevés par les experts ont aggravé les conséquences dommageables de l’infection nosocomiale dont M. A... a été victime, ainsi que l’ont relevé les premiers juges, et sont constitutifs de fautes de nature à engager la responsabilité du centre hospitalier universitaire de Clermont Ferrand.

6.	Enfin, il résulte de l’instruction et notamment du rapport d’expertise du professeur Massip et du docteur Lignac, sans que cela ne soit contesté, que la prise en charge de M. A... par le docteur Béal de la clinique « Pôle Santé République » à compter du 14 décembre 2004 n’a pas été conforme aux règles de l’art dès lors qu’après l’ablation du matériel le 3 mai, les foyers osseux n’ont pas été rigoureusement immobilisés et que le traitement médical de l’infection a été insuffisant. 

7.	Les experts concluent que les manquements du centre hospitalier universitaire et du docteur Béal de la clinique du « Pôle Santé République » ont fait perdre une chance de guérison plus rapide au patient. 

8.	Les premier juges ont retenu qu’« eu égard à l’ensemble des circonstances et des fautes respectives commises par le centre hospitalier universitaire de Clermont Ferrand et le docteur Béal qui ont concouru à la survenance et à l’aggravation du dommage, il y a lieu de mettre à la charge du centre hospitalier universitaire de Clermont Ferrand 75 % de l’indemnité due aux consorts A.... » Ce taux de 75 % n’étant pas contesté en appel, il y a lieu de maintenir à la charge du centre hospitalier universitaire de Clermont Ferrand la réparation de 75 % des conséquences dommageables résultant de l’infection nosocomiale et des manquements commis par le centre hospitalier universitaire. 


Sur l’évaluation des préjudices de M. A... : 

9.	Aucune des parties ne conteste le jugement attaqué en ce qu’il a fixé, d’une part les préjudices à caractère extrapatrimonial de M. A..., à la somme de 8 656,12 euros au titre du déficit fonctionnel temporaire, à la somme de 18 150 euros au titre du déficit fonctionnel permanent, à la somme de 2 722,50 euros au titre du préjudice d’agrément, à la somme de 1 725 euros au titre du préjudice esthétique permanent, d’autre part en ce qu’il a fixé, le préjudice à caractère patrimonial de M. A..., au titre de son préjudice de formation à la somme de 2 250 euros. 

En ce qui concerne la perte de revenus :

10.	M. A... fait valoir qu’au moment de la survenance de l’infection, il était âgé de 17 ans et était employé en qualité d’apprenti pâtissier et que l’infection nosocomiale dont il a été victime et les défaillances dans sa prise en charge l’ont privé de tous gains professionnels pendant de nombreuses années.

11.	Il résulte de l’instruction qu’à la date de l’accident le 4 septembre 2004, M. A..., alors âgé de 17 ans, suivait une formation au certificat d’aptitude professionnelle de pâtisserie au centre de formation des apprentis de Clermont-Ferrand et avait conclu un contrat d’apprentissage le 23 juillet 2004 avec la société La Panifière, mais qui avait commencé à être exécuté le 12 juillet 2004, et pour lequel il percevait 25 % du salaire minimum interprofessionnel de croissance (SMIC) au titre de la première année. Le 6 septembre 2004, le père de M. G... A... a informé l’employeur de son fils qu’à la suite de son accident de la circulation, il était raisonnable de ne pas confirmer son contrat d’apprentissage dans la mesure où il ne pouvait pas suivre les cours du centre de formation des apprentis tout comme il ne pouvait apprendre son métier au sein de la société La Panifière en raison d’une longue absence. Le même jour, il a annulé l’inscription de son fils en première année du certificat d’aptitude professionnelle. Il s’ensuit que la rupture du contrat d’apprentissage résulte, non de l’infection nosocomiale dont les premiers signes évocateurs sont apparus le 13 septembre, mais de la nécessaire prise en charge de la double fracture fermée du fémur droit dont M. G... A... a été victime à la suite de son accident de la circulation.

12.	Si l’employeur de M. G... A... a indiqué, dans une attestation du 21 février 2012 soit huit ans après l’accident, qu’il était satisfait du travail de son apprenti et pensait qu’il pourrait poursuivre sa formation dans l’entreprise après son rétablissement, il résulte de l’instruction que M. A... avait commencé son apprentissage en pâtisserie, après avoir suivi une première année de formation comme dessinateur, depuis un peu moins de deux mois avant son accident et qu’il aurait subi une période de déficit fonctionnel temporaire en lien uniquement avec l’accident de sept mois. Par suite, il n’est pas établi qu’à la suite de la période de convalescence résultant des conséquences séquellaires de l’accident dont M. A... a été victime, celui-ci aurait été repris en contrat d’apprentissage par son ancien employeur. 

13.	M. A... fait également valoir qu’à l’issue de sa période de formation, il aurait alors été titulaire d’un certificat d’aptitude professionnel de pâtissier et qu’il aurait pu occuper un emploi d’ouvrier de catégorie 2. Toutefois, en l’absence de tout élément permettant d’apprécier les capacités de M. A... à obtenir son certificat d’aptitude professionnel en pâtisserie, celui ci s’étant engagé dans cette formation peu de temps avant son accident, il n’est pas établi que la perte de revenus alléguée présenterait un caractère certain ouvrant droit à une indemnisation. Il s’ensuit que ce chef de préjudice ne peut être accueilli.

En ce qui concerne l’incidence professionnelle :

14.	M. A... fait valoir qu’il a subi, du fait des préjudices en lien avec l’infection nosocomiale et ses suites, une importante dévalorisation sur le marché du travail et qu’il a été contraint de s’orienter vers le métier d’auxiliaire en prothèse dentaire qui s’est avéré inadapté à son handicap.

15.	Il résulte de l’instruction et notamment du rapport d’expertise que le déficit fonctionnel permanent dont est atteint M. A... « doit prendre en compte le raccourcissement de longueur du membre inférieur droit supérieur à 3 cm et la pathologie du genou droit (raideur du genou dont la flexion ne dépasse pas 45°, instabilité du genou avec dérobements, syndrome douloureux). Cette pathologie séquellaire du genou n’est pas en totalité imputable au dommage causé par l’infection nosocomiale car existe une part en rapport avec les séquelles des lésions trochléennes contemporaines des foyers de fracture ». Les experts ont estimé que le déficit fonctionnel permanent en rapport avec l’infection nosocomiale devait être estimé à 15 %. Par une décision du 29 mai 2012, la maison départementale des personnes handicapées de la Haute-Garonne a reconnu M. G... A... en qualité de travailleur handicapé à compter du 1er octobre 2012. M. A... a été contraint de se réorienter professionnellement vers le métier d’auxiliaire en prothèse dentaire. S’il a été embauché comme employé en prothèse dentaire le 12 septembre 2016, son employeur l’a licencié en juin 2018 pour inaptitude définitive à son poste d’auxiliaire en prothèse dentaire conformément à l’avis du médecin du travail.

16.	Compte tenu de l’âge de M. A... et de la durée du déficit fonctionnel temporaire qui a retardé sa formation professionnelle, de l’obligation dans laquelle il s’est trouvé en raison des séquelles résultant de l’infection nosocomiale de procéder à un changement dans l’orientation professionnelle dans laquelle il s’était engagé depuis peu, de son licenciement pour inaptitude définitive à son poste d’auxiliaire en prothèse dentaire en raison de la nécessité d’occuper un poste exclusivement assis, il sera fait une juste appréciation de ce chef de préjudice en l’évaluant à la somme de 38 000 euros.

17.	Aux termes de l’article L. 376 1 du code de la sécurité sociale, « Lorsque, sans entrer dans les cas régis par les dispositions législatives applicables aux accidents du travail, la lésion dont l'assuré social ou son ayant droit est atteint est imputable à un tiers, l'assuré ou ses ayants droit conserve contre l'auteur de l'accident le droit de demander la réparation du préjudice causé, conformément aux règles du droit commun, dans la mesure où ce préjudice n'est pas réparé par application du présent livre ou du livre Ier. / Les caisses de sécurité sociale sont tenues de servir à l'assuré ou à ses ayants droit les prestations prévues par le présent livre et le livre Ier, sauf recours de leur part contre l'auteur responsable de l'accident dans les conditions ci-après. /Les recours subrogatoires des caisses contre les tiers s'exercent poste par poste sur les seules indemnités qui réparent des préjudices qu'elles ont pris en charge, à l'exclusion des préjudices à caractère personnel. »

18.	Si M. A... indique qu’il convient de ne pas déduire de l’indemnité allouée au titre de l’incidence professionnelle la somme de 17 583,81 euros correspondant aux frais de reclassement professionnel versés par la caisse primaire d’assurance maladie de la Haute Garonne au Centre de Rééducation des Invalides Civils, le préjudice d’incidence professionnelle inclut les frais de reclassement professionnel et de formation. Par suite, il y a lieu de déduire du montant de l’indemnité allouée au titre de l’incidence professionnelle les sommes perçues au titre des frais de formation. Compte tenu de l’évaluation de ce chef de préjudice à la somme de 38 000 euros et du taux de perte de chance retenu de 75 % imputable au centre hospitalier, le montant maximal de l’indemnisation pouvant être mis à la charge du centre hospitalier universitaire de Clermont Ferrand est de 28 500 euros. Il résulte du relevé des débours de la caisse primaire d’assurance maladie que M. A... a perçu la somme de 17 788,46 euros au titre des frais de formation engagés du 19 mars 2012 au 30 avril 2013. Il y a lieu, par suite, de mettre à la charge du centre hospitalier universitaire de Clermont Ferrand la somme de 20 211,54 euros (soit 38 000 euros moins 17 788,46 euros) qui n’excède pas le montant de l’indemnisation susceptible d’être mise à la charge du centre hospitalier universitaire. 

En ce qui concerne les souffrances endurées :

19.	Il résulte de l’instruction et notamment du rapport d’expertise que l’expert a fixé ce chef de préjudice à 6 sur une échelle de 7 en retenant la longueur d’évolution, le nombre élevé d’interventions et la longueur des hospitalisations. En évaluant à la somme de 20 000 euros les souffrances endurées, soit après application du taux de perte de chance la somme de 15 000 euros, les premiers juges ont fait une évaluation qui n’est ni insuffisante ni excessive.

20.	Il résulte de tout ce qui précède que M. A... est seulement fondé à demander que la somme de 15 312 euros qui a été mise à la charge du centre hospitalier universitaire de Clermont Ferrand par le jugement du 6 mars 2019 du tribunal administratif de Clermont Ferrand au titre de l’incidence professionnelle soit portée à la somme de 20 211,54 euros. 

Sur l’indemnité forfaitaire de gestion :

21.	La caisse primaire d’assurance maladie de la Haute-Garonne, qui se borne à demander la confirmation du jugement en tant qu’il lui a accordé satisfaction, n’obtient ni ne sollicite devant la cour aucune somme complémentaire au titre du remboursement de ses débours. Par suite, elle n’est pas fondée à demander que l’indemnité forfaitaire de gestion prévue par les dispositions de l’article L. 376 1 du code de la sécurité sociale, qui lui a été attribuée à hauteur de 1 080 euros en vertu de l’arrêté du 27 décembre 2018 relatif aux montants de l'indemnité forfaitaire de gestion prévue aux articles L. 376 1 et L. 454 1 du code de la sécurité sociale alors applicable, soit rehaussée pour tenir compte du montant revalorisé de cette indemnité à la date du présent arrêt.

Sur les intérêts et leur capitalisation :

22.	M. A... a droit aux intérêts légaux sur la somme de 20 211,54 à compter de 27 juin 2012, date de la saisine de la commission régionale de conciliation et d’indemnisation. M. A... a également droit à la capitalisation annuelle des intérêts à compter du 11 octobre 2016 puis à chaque échéance annuelle ultérieure.

Sur les frais liés au litige : 

23.	Il y a lieu, dans les circonstances de l’espèce, de mettre à la charge du centre hospitalier universitaire de Clermont Ferrand la somme de 1 500 euros au titre des frais exposés par M. A... et non compris dans les dépens.

24.	Il n’y a pas lieu, dans les circonstances de l’espèce, de faire droit aux conclusions présentées par la caisse primaire d’assurance maladie de la Haute Garonne au titre des dispositions de l’article L. 761 1 du code de justice administrative.



D E C I D E  :

Article 1er : La somme de 15 312 euros qui a été mise à la charge du centre hospitalier universitaire de Clermont Ferrand par le jugement du 6 mars 2019 du tribunal administratif de Clermont Ferrand au titre de l’incidence professionnelle est portée à la somme de 20 211,54 euros. 

Article 2 : La somme de 20 211,54 euros portera intérêts à compter du 27 juin 2012. Les intérêts échus le 11 octobre 2016, puis à chaque échéance annuelle à compter de cette date, seront capitalisés à chacune de ces dates pour produire eux mêmes intérêts.

Article 3 : Le jugement du 6 mars 2019 du tribunal administratif de Clermont Ferrand est réformé en ce qu’il a de contraire au présent arrêt.

Article 4 : Le centre hospitalier universitaire de Clermont Ferrand versera la somme de 1 500 euros à M. A... en application des dispositions de l’article L. 761 1 du code de justice administrative.

Article 5 : Le surplus des conclusions de la requête de M. A... et les conclusions de la caisse primaire d’assurance maladie de la Haute Garonne sont rejetés. 

Article 6 : Le présent arrêt sera notifié à M. G... A..., à la caisse primaire d'assurance maladie du Puy de Dôme, à la caisse primaire d'assurance maladie de la Haute Garonne, à M. H... et Mme D... A... et au centre hospitalier universitaire de Clermont Ferrand.


Délibéré après l'audience du 25 mars 2021, à laquelle siégeaient :

M. Pourny, président de chambre,
M. Gayrard, président assesseur,
Mme B..., première conseillère. 


Rendu public par mise à disposition au greffe le 16 avril 2021.

"""

In [23]:
start_line = "Considérant ce qui suit :"
end_line = "D E C I D E  :"
text_motif = extract_decision_component(text_decision, start_line, end_line)

In [25]:
#text_motif

In [26]:
start_line = "D E C I D E  :"
end_line = None
text_decide = extract_decision_component(text_decision, start_line, end_line)

In [28]:
#text_decide

In [29]:
data = {
    "decision_complete": text_decision,
    "partie_motif": text_motif,
    "partie_resultat": text_decide
    
}

In [30]:
synthesis = {
    "litige":
    {
        "qui sont les ou le requérant(s) ?": "",
        "Qui a produit un mémoire en défense?": "",
        "De quoi le requérant se plaint-il (sans donner les arguments)?":"",
        "Le requérant veut-il que le juge annule une décision ou lui octroie une somme d’argent ?": ""
    },
    "result":
    {
        "le requérant a-t-il  gagné ou perdu au regard du contexte ?":"",
        "Des sommes ont-elles été versées? Quel montant?":""
        
    },
    "motif":
    {
        "Pourquoi le requérant a-t-il échoué/réussi/non-lieu à statuer au regard du contexte (attention aux bonnes juridictions)?":""  
    }
    
}

# Production des synthèse

In [31]:
# Traitement du titre
import time
start_time = time.time()
synthesis["Titre"] = explication(data["decision_complete"], prompt_titre)
print("--- %s seconds ---" % (time.time() - start_time))

--- 5.121873140335083 seconds ---


In [32]:
# Traitement de la partie litige
import time
start_time = time.time()

for query in synthesis["litige"]:
    synthesis["litige"][query] = extract(data["decision_complete"], query)

answers_concatenated = "\n".join(list(synthesis["litige"].values()))

for p in persona:
    synthesis["litige"][p] = explication(answers_concatenated, persona[p])
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 75.79789900779724 seconds ---


In [33]:
# Traitement de la partie résultat
import time
start_time = time.time()

for query in synthesis["result"]:
    synthesis["result"][query] = extract(data["partie_resultat"], query)

answers_concatenated = "\n".join(list(synthesis["result"].values()))

for p in persona:
    synthesis["result"][p] = explication(answers_concatenated, persona[p])
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 64.60607028007507 seconds ---


In [34]:
# Traitement de la partie motif
import time
start_time = time.time()

for query in synthesis["motif"]:
    synthesis["motif"][query] = extract(data["partie_motif"], query)

answers_concatenated = "\n".join(list(synthesis["motif"].values()))

for p in persona:
    synthesis["motif"][p] = explication(answers_concatenated, persona[p])
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 120.94379734992981 seconds ---


In [35]:
# Traitement du resumé
import time
synthesis["Resumé"] = {"Patrice":"",
                      "Anna":"",
                      "Hugo":""}


start_time = time.time()
for p in persona:
    
    if p == "Patrice":
        prompt = prompt_resume_patrice
    elif p == "Anna":
        prompt = prompt_resume_anna
    elif p == "Hugo":
        prompt = prompt_resume_hugo
    
    list_synthesis = []
    for decision_p in synthesis:
        if decision_p != "Titre":
            list_synthesis.append(synthesis[decision_p][p])
        
    text = ". ".join(list_synthesis)
    
    synthesis["Resumé"][p] = explication(text, prompt)
print("--- %s seconds ---" % (time.time() - start_time))

--- 25.70161724090576 seconds ---


In [40]:
synthesis["Resumé"]["Anna"]

"Monsieur A a eu des problèmes de santé à cause d'un hôpital et a perdu son travail. Il a demandé de l'argent pour cela. Au début, il a reçu 15 312 euros, mais il voulait plus. Il a fait appel et la cour a augmenté la somme à 20 211,54 euros, plus 1 500 euros pour les frais de justice. L'hôpital doit payer 75% de cette somme. Monsieur A n'a pas eu d'argent pour la perte de salaire car il n'a pas pu prouver que c'était à cause de l'hôpital."

# Sauvegarde des synthèses

In [ ]:
import pickle
pickle.dump(synthesis, open( "synthesis_1804_v1.p", "wb" ) )

# Affichage des synthèse

## Lecture de la synthèse

In [ ]:
synthesis_loaded = pickle.load( open( "synthesis_personna_v0925.p", "rb" ) )

In [ ]:
synthesis_loaded["Resumé"]["Hugo"]

## Objet de Litige

In [ ]:
for key in synthesis_loaded["litige"]:
    print("--------------- Question -----------------------")
    print(key)
    print("******* Réponse ********")
    print(synthesis_loaded["litige"][key])

## Résultat

In [ ]:
for key in synthesis_loaded["result"]:
    print(synthesis_loaded["result"][key])
    print("----------------------------------------------------")

## Motif

In [ ]:
for key in synthesis_loaded["motif"]:
    print(synthesis_loaded["motif"][key])
    print("----------------------------------------------------")